In [1]:
import sys, os, json, pickle, gzip
sys.path.append('..')
#from src.data import BaseDataset, ConditionalData
import numpy as np
import gzip, pickle
from src.utils import SortedAllElements, Element

# ELMD

In [2]:
with gzip.open('../data/unique_data.pkl.gz','rb') as f:
    unique_data = pickle.load(f)
target_elements = np.zeros(len(SortedAllElements))
for data in unique_data:
    for ele, frac in data['target_comp'].items():
        target_elements[Element(ele).number] += data['count']
target_em = target_elements.astype(bool)

In [49]:
elmd = {}
for fn in os.listdir('../src/elmd'):
    if not fn.endswith('json'): continue
    elmd[fn.split('.')[0]] = json.load(open(f'../src/elmd/{fn}'))

for fn, data in sorted(elmd.items(), key=lambda x: x[0]):
    dtype = []
    dlen = []
    print(fn)
    ele_not_in_data = []
    ele_not_in_table = []
    for ele, feat in data.items():
        dtype.append(str(type(feat)).split()[1].replace('>','').replace('\'',''))
        if isinstance(feat, (int, float)):
            dlen.append(1)
        else:
            dlen.append(len(feat))
        if ele not in SortedAllElements:
            ele_not_in_table.append(ele)
    for ele in np.array(SortedAllElements)[target_em][1:]:
        if ele not in data.keys():
            ele_not_in_data.append(ele)
    print(dtype[0], dlen[0], len(dtype))
    print(ele_not_in_data)
    print(ele_not_in_table)
    print()
    

atomic
int 1 121
[]
['D', 'T', 'Uue']

cgcnn
list 91 100
[]
[]

elemnet
list 135 98
[]
[]

jarvis
list 438 82
['Eu', 'Np', 'Pu', 'Am', 'Cm']
[]

jarvis_sc
list 438 82
['Eu', 'Np', 'Pu', 'Am', 'Cm']
[]

magpie
list 21 97
[]
[]

magpie_sc
list 22 97
[]
[]

mat2vec
list 199 118
[]
[]

matscholar
list 200 103
[]
[]

megnet16
list 16 95
['Am', 'Cm']
['Null']

mendeleev
int 1 121
[]
['D', 'T', 'Uue']

mod_petti
int 1 121
[]
['D', 'T', 'Uue']

oliynyk
list 44 85
['Np', 'Pu', 'Am', 'Cm']
[]

oliynyk_sc
list 44 85
['Np', 'Pu', 'Am', 'Cm']
[]

petti
int 1 121
[]
['D', 'T', 'Uue']

random_200
list 200 82
['Eu', 'Np', 'Pu', 'Am', 'Cm']
[]



In [6]:
from src.feature import composition_to_feature

composition_to_feature({'Li':0.2, 'Co':0.2, 'O':0.6}, feature_type='magpie')

array([6.40000000e+01, 2.27744790e+01, 4.77218000e+02, 1.16000000e+01,
       2.40000000e+00, 9.04000000e+01, 2.63600000e+00, 1.80000000e+00,
       2.40000000e+00, 1.40000000e+00, 0.00000000e+00, 5.60000000e+00,
       2.00000000e-01, 1.20000000e+00, 6.00000000e-01, 0.00000000e+00,
       2.00000000e+00, 1.08306667e+01, 0.00000000e+00, 3.09694240e-01,
       9.18000000e+01])